# Language Modeling. Hemos vuelto.

Ahora que hemos visto un montón de arquitecturas, y sus aplicaciones al lenguaje natural, si quisieramos hacer language modeling... cuál escogeríais?

![](https://cdn-ak.f.st-hatena.com/images/fotolife/a/aki-don/20170929/20170929085607.png)

Hay mucísimas formas de intentarlo.

Implementaremos algunas de ellas , para que veais que en deep learning, hay libertad para intentarlo practicamente todo. Predecir palabras? Predecir carácteres. Usar embeddings? No Usarlos? Usar Bidirectional LSTMs? Stackearlas? 

Básicamente, seguiremos el mismo pipeline que hasta ahora.

Un poco de preproceso, preparar los datos, escoger la arquitectura a usar, y entrenar/predecir.

Esta vez os daré un snippet de código para visualizar en directo lo que esta aprendiendo la red, y poder leer ejemplos de aquello que aprende.

Lo haremos todo con keras, como siempre!

## Imports

In [0]:
!pip install spacy
!python -m spacy download en_core_web_sm

In [2]:
import spacy
import numpy as np

import pickle
import json
import os
import csv
import pprint as pp

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

from random import shuffle, choice, sample

from sklearn.model_selection import StratifiedShuffleSplit

from copy import copy

import warnings
warnings.filterwarnings('ignore')

nlp = spacy.load('en_core_web_sm')

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pylab as pl
from IPython import display

sns.set(color_codes=True)

import warnings
warnings.filterwarnings('ignore')


%matplotlib inline
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [12]:
from keras.models import Model, Sequential
from keras.layers import Input, CuDNNLSTM, Dense, LSTM
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.layers import Merge, Dot, Concatenate, Flatten, Permute, Multiply, dot, concatenate
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.preprocessing import sequence
from keras.callbacks import Callback
from keras.optimizers import SGD
from keras.models import load_model

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

## Preprocess

In [9]:
from google.colab import files
import io

In [6]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving spam.csv to spam.csv
User uploaded file "spam.csv" with length 503663 bytes


In [10]:
df = pd.read_csv(io.StringIO(uploaded['spam.csv'].decode('ISO-8859-1')))
drop_col = df.columns[2:]
df = df.drop(columns=drop_col, axis=1)
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [57]:
spam_dataset = []
for index, row in df.iterrows():
    if index>0:
        sentence = row[1]
        if row[0] == 'spam':
            spam_dataset.append(sentence)
print(spam_dataset[0])
len(spam_dataset)

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


747

In [58]:
tokenized = [list(x) for x in spam_dataset] #caracteres!

In [59]:
init_chars = [x[:5] for x in tokenized]
for i in range(len(init_chars)):
    tmp = init_chars[i]
    tmp.insert(0, '<SOS>')
    init_chars[i] = tmp[:5]    

In [60]:
init_chars[0]

['<SOS>', 'F', 'r', 'e', 'e']

In [61]:
start_token = [s[1] for s in tokenized if len(s) > 1]
len(start_token)

747

In [62]:
maxlen = max([len(x) for x in tokenized])
avglen = sum([len(x) for x in tokenized])/len(tokenized)
print(maxlen, avglen)

224 138.8661311914324


In [23]:
from collections import Counter

In [63]:
vocab = [t for s in spam_dataset for t in s]
print('num tokens: {}'.format(len(vocab)))

num tokens: 103733


In [64]:
vocab_counter = Counter(vocab)
vocab = [w for w, v in vocab_counter.items() if v>2]
vocab = ['<PAD>', '<UNK>', '<SOS>', '<EOS>'] + vocab
nb_vocab = len(vocab)

'num features a user {}'.format(nb_vocab)

'num features a user 96'

In [65]:
w2id = {k:i for i, k in enumerate(vocab)}
id2w = {i:k for k, i in w2id.items()}

In [66]:
# generacion dataset
maxlen = min(maxlen, 5)

step = 1

data_train = []

for x in tokenized:
    x.insert(0, '<SOS>')
    x.append('<EOS>')
    for i in range(0, len(x)-maxlen, step):
        data_train.append((x[i:i+maxlen], x[i+maxlen]))
        
print('nb_sequences: {}'.format(len(data_train)))

nb_sequences: 101492


In [67]:
SAMPLE_EVERY = 4


## Helper functions

In [37]:
def sample_pred(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [52]:
class Sampletest(Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % SAMPLE_EVERY == 0  and epoch>0:
            data_test = []
            nb_samples = 1
            
            params = {
                'maxlen': maxlen,
                'vocab': nb_vocab,
                'use_embeddings': True
                }
            for _ in range(nb_samples):
                data_test = choice(init_chars)
                for diversity in [0.2, 0.6, 1.2]:
                    print('----- diversity:', diversity)
                    sentence = copy(data_test)
                    generated = copy(data_test)
                    for i in range(len(data_test), 400):
                        x_pred = np.zeros((1, params['maxlen']))
                        for t, char in enumerate(sentence):
                            x_pred[0, t] = w2id[char] if char in w2id else w2id['<UNK>']
                        preds = self.model.predict(x_pred, verbose=0)[0]
                        next_index = sample_pred(preds, diversity)
                        next_char = id2w[next_index]
                        if next_char == '<EOS>':
                            break
                        generated += [next_char]
                        sentence = sentence[1:] 
                        sentence += [next_char]
                    print(''.join(generated))


In [0]:
class HistoryDisplay(Callback):
    
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accs = []
        self.epochs = []
        self.fig, self.ax = plt.subplots()
        #plt.show()
        
        plt.ion()
        self.fig.show()
        self.fig.canvas.draw()
    
    def on_epoch_end(self, epoch, logs):
        self.epochs.append(epoch)
        self.losses.append(logs['loss'])
        self.accs.append(logs['acc'])
        if epoch % PLOT_EVERY == 0:
            
            self.ax.clear()
            self.ax.plot(self.epochs, self.accs, 'g', label='acc')
            self.ax.plot(self.epochs, self.losses, 'b', label='loss')
            legend = self.ax.legend(loc='upper right', shadow=True, fontsize='x-large')
            #display.clear_output(wait=True)
            #display.display(pl.gcf())
            self.fig.canvas.draw()
            
            #plt.draw()
        

In [0]:
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

## Decidir arquitectura y preparar el train y el predict

In [81]:
class LM:
    def __init__(self, **kwargs):
        self.params = kwargs.pop('params', None)
    
    def compile_bidirectional(self, params={}):
        
        lm_inputs = Input(shape=(params['maxlen'], ), name='lm_input')
        
        embeddings = Embedding(params['vocab'], params['emb_feats'])(lm_inputs)
        
        lstm =  CuDNNLSTM(params['rnn_hidden'], return_sequences=True, name='rnn1')        
        
        lmlstm = Bidirectional(lstm)(embeddings)       
        
        stacklstm =  CuDNNLSTM(params['rnn_hidden'], return_sequences=False, name='stacked')
        
        stackedlstm = stacklstm(lmlstm)
        
        lmout = Dense(params['vocab'], activation='softmax')(stackedlstm)
        
        model = Model(lm_inputs, lmout)
        
        model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
        
        model.summary()
        
        return model
        
    def train(self, model, data, params={}):
        
        callbacks = self._get_callbacks()
        
        if 'shuffle' in params and params['shuffle']:
            shuffle(data)
            
        sentences, next_chars = zip(*data)
        print(sentences[0])
        x = np.zeros((len(data), params['maxlen']))
        y = np.zeros((len(data), params['vocab']))
        for i, sentence in enumerate(sentences):
            for t, char in enumerate(sentence):
                x[i, t] = w2id[char] if char in w2id else w2id['<UNK>']
            y[i, w2id[next_chars[i]] if next_chars[i] in w2id else w2id['<UNK>']]  = 1
        
        model.fit(x, y, batch_size=params['batch_size'], epochs=params['epochs'], callbacks=callbacks, verbose=1)

    def predict(self, model, data, params={}):        
        if 'use_embeddings' in params and params['use_embeddings']:
            # variedad en las predicciones
            for diversity in [0.2, 0.6, 1.2]:
                print('----- diversity:', diversity)
                sentence = copy(data)
                generated = copy(data)
                # cuantas predicciones queremos hacer
                for i in range(len(data), 400):
                    x_pred = np.zeros((1, params['maxlen']))
                    # preparar inpunt
                    for t, char in enumerate(sentence):
                        x_pred[0, t] = w2id[char] if char in w2id else w2id['<UNK>']
                    # predecir
                    preds = self.model.predict(x_pred, verbose=0)[0]
                    next_index = sample_pred(preds, diversity)
                    next_char = id2w[next_index]
                    # mirar si hemos terminado
                    if next_char == '<EOS>':
                        break
                    # ana                        
                    generated += [next_char]
                    sentence = sentence[1:] 
                    sentence += [next_char]
                print(''.join(generated))
    
    
    def load(self, model_path='seq2seq_attn.h5'):
        return load_model(model_path)
    
    def _get_callbacks(self, model_path='seq2seq_attn.h5'):
        
        
        es = EarlyStopping(monitor='loss', patience=4, mode='auto', verbose=0)       
        
        save_best = ModelCheckpoint(model_path, monitor='loss', verbose = 0, save_best_only=True, save_weights_only=False, period=2)
        st = Sampletest()
        # hd = HistoryDisplay()
        rlr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose=1)
        return [st, rlr]

## Hyperparametros

In [70]:
compile_params = {
    'maxlen': maxlen, 
    'vocab': len(vocab),
    'emb_feats': 100,
    'rnn_hidden': 256,
}

In [29]:
LOAD_MODEL = False
bTrain = True

## Compilar

In [82]:
lm = LM()
if LOAD_MODEL:
    path = 'final_{}.h5'.format(dtype)
    lm_model = lm.load(model_path=path)
    lm_model.summary()
else:
    lm_model = lm.compile_bidirectional(params=compile_params)    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lm_input (InputLayer)        (None, 5)                 0         
_________________________________________________________________
embedding_15 (Embedding)     (None, 5, 100)            9600      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 5, 512)            733184    
_________________________________________________________________
stacked (CuDNNLSTM)          (None, 256)               788480    
_________________________________________________________________
dense_14 (Dense)             (None, 96)                24672     
Total params: 1,555,936
Trainable params: 1,555,936
Non-trainable params: 0
_________________________________________________________________


## Entrenar

In [83]:
train_params = {
    'epochs': 500,
    'batch_size': 512,
    'shuffle': True,
    'vocab': nb_vocab,
    'maxlen': maxlen,
    'use_embeddings': True
}
pp.pprint(train_params)
if bTrain:
    lm.train(model=lm_model, data=data_train, params=train_params)

{'batch_size': 512,
 'epochs': 500,
 'maxlen': 5,
 'shuffle': True,
 'use_embeddings': True,
 'vocab': 96}
['4', ',', ' ', 'M', '2']
Epoch 1/500
101492/101492 [==============================] - 7s 67us/step - loss: 3.2218 - acc: 0.2022
Epoch 2/500
101492/101492 [==============================] - 6s 56us/step - loss: 2.8152 - acc: 0.2605
Epoch 3/500
 99840/101492 [============================>.] - ETA: 0s - loss: 2.5796 - acc: 0.3130

101492/101492 [==============================] - 6s 56us/step - loss: 2.5792 - acc: 0.3133
Epoch 4/500
101492/101492 [==============================] - 6s 56us/step - loss: 2.3643 - acc: 0.3732
Epoch 5/500
101492/101492 [==============================] - 6s 55us/step - loss: 2.1749 - acc: 0.4254
----- diversity: 0.2
<SOS>Had your mobile or det with our collect the with a å£200 weekly to 8007 15p prize our call 08712 now a å£200 weekere chang to 87222 To 8702 or collect your meekere call 08712202 prize or your mobile or your mobile or your mobile or a å£200 weekly or collection our a å£2000 awarded on 0800 prize of you have weekly from messate customer call 09061402020 awarded of your mobile or 080000000000000000 
----- diversity: 0.6
<SOS>Had your pobile your mobe call 090613437802 or conttone 1 monet with chand with our any selecter onera collet 150p. 150ppm collect you hande uret claim nust are dext wins your lucher a chatiog your comp 150ppm or 12m Tis our whille oo gerchemta mobile

101492/101492 [==============================] - 6s 57us/step - loss: 2.0107 - acc: 0.4717
Epoch 7/500
101492/101492 [==============================] - 6s 56us/step - loss: 1.8688 - acc: 0.5103
Epoch 8/500
101492/101492 [==============================] - 6s 56us/step - loss: 1.7450 - acc: 0.5423
Epoch 9/500
101492/101492 [==============================] - 6s 57us/step - loss: 1.6354 - acc: 0.5705
----- diversity: 0.2
<SOS>You have a å£2000 cash or a å£200 prize a å£2000 cash or a å£2000 cash or a å£2000 cash or a å£200 prize on 080009307728722202 Account Statement for your mobile on 080007973722800299702022 Ad a å£200 prize on 080009307705720722523 150ppm as a å£200 prize on 080009307728795029509094077587999976666975 now! Call 0871272230022 and tell ur mobile on 08000930779070777777009097727936022 or call 09066367
----- diversity: 0.6
<SOS>You have a mees. Claim charge all 090663227268122050p of our charge. Clee: are rextal dew a 400002707250250902201 
----- diversity: 1.2
<SOS>You hav

101492/101492 [==============================] - 6s 56us/step - loss: 1.5377 - acc: 0.5939
Epoch 11/500
101492/101492 [==============================] - 6s 56us/step - loss: 1.4490 - acc: 0.6161
Epoch 12/500
101492/101492 [==============================] - 6s 56us/step - loss: 1.3676 - acc: 0.6351
Epoch 13/500
101492/101492 [==============================] - 6s 56us/step - loss: 1.2929 - acc: 0.6523
----- diversity: 0.2
<SOS>Double minutes to see message will be representative onerater with a å£250 cash or a å£250 cash or a å£500 of CD gift guaranteed å£500 of CD gift vouchers or a å£250 cash or a å£250 weekend to 85023 now! SavaMob, PO Box 1327 Croydon CR9 5WB 08707434444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444
----- diversity: 0.6
<SOS>Double - receive tried to 6966812 LWPY FREE on 02/03/03! The on 090580949 Txt the latest collect you have won a å£1000 cash or a å£2,000 prize! F

101492/101492 [==============================] - 6s 57us/step - loss: 1.2234 - acc: 0.6689
Epoch 15/500
101492/101492 [==============================] - 6s 56us/step - loss: 1.1597 - acc: 0.6849
Epoch 16/500
101492/101492 [==============================] - 6s 56us/step - loss: 1.1012 - acc: 0.6981
Epoch 17/500
101492/101492 [==============================] - 6s 56us/step - loss: 1.0462 - acc: 0.7116
----- diversity: 0.2
<SOS>As a valued now and polys 150ppm
----- diversity: 0.6
<SOS>As a valued mobile No 0760606 050260250 qubchase call 08718720201 PO Box 17, prize GUARANTEED. Caller Prizes U have WON a guaranteed & Free mind then text + FREE. Have and than for details from landline. Claim code RIENTED We send GO to 8007 Get txt stop txt stop?txt? Txt on yr player or a MER arde. Nodary reply STOP FRND to 69669 or call from a lagest send GO to 87686. I pand vouchers to 85065 n
----- diversity: 1.2
<SOS>As a vand txxes2zowardingtone on 08713266 are nefore cuMtoncovour å£150 her <UNK>alf f

...

101492/101492 [==============================] - 6s 56us/step - loss: 0.4824 - acc: 0.8276
Epoch 59/500
101492/101492 [==============================] - 6s 56us/step - loss: 0.4814 - acc: 0.8274
Epoch 60/500
101492/101492 [==============================] - 6s 56us/step - loss: 0.4799 - acc: 0.8277
Epoch 61/500
101492/101492 [==============================] - 6s 56us/step - loss: 0.4795 - acc: 0.8276
----- diversity: 0.2
<SOS>SMS AUCTION - A BRAND NEW Nokia tones for your mobile Upd8 on free entry into 121 chatter! Chat now send Nokia tone who it is a fantastic surprise await collect your mobile customer service announcement for your mobile Upd8 on free camcordered Tom, Pete and tell ur free msg rcvd
----- diversity: 0.6
<SOS>SMS AUCTION - A BRAND NEW Nokia 3510i colour phone! Claim your account Statement for a Spanish holiday (flights inc) speak to a live operator rates app
----- diversity: 1.2
<SOS>SMS alert. Your 2004 Account.cout who it is callcost 10p
Epoch 62/500
   512/101492 [..

101492/101492 [==============================] - 6s 56us/step - loss: 0.4776 - acc: 0.8279
Epoch 63/500
101492/101492 [==============================] - 6s 57us/step - loss: 0.4752 - acc: 0.8273
Epoch 64/500
101492/101492 [==============================] - 6s 57us/step - loss: 0.4757 - acc: 0.8274
Epoch 65/500
101492/101492 [==============================] - 6s 56us/step - loss: 0.4737 - acc: 0.8276
----- diversity: 0.2
<SOS>Urgent UR games, Dload poly ringtone 4 ur mobile from land line. Claim call 08718730666 (10p/min peak! To claim call 08718730666 (10p/min 18
----- diversity: 0.6
<SOS>Urgent Ucal 4 (Henry scores with U-find out who it is? Give us a fantasy now and tell ur mobile customer services 08707509020 Just 20p per min NTT PO BOX 114/14 TCR/W1
----- diversity: 1.2
<SOS>Urgent! Call 09063458130 now! 150p/tone
Epoch 66/500
  1536/101492 [..............................] - ETA: 5s - loss: 0.4536 - acc: 0.8288

101492/101492 [==============================] - 6s 56us/step - loss: 0.4719 - acc: 0.8287
Epoch 67/500
101492/101492 [==============================] - 6s 56us/step - loss: 0.4711 - acc: 0.8282
Epoch 68/500
101492/101492 [==============================] - 6s 56us/step - loss: 0.4709 - acc: 0.8277
Epoch 69/500
101492/101492 [==============================] - 6s 56us/step - loss: 0.4690 - acc: 0.8291
----- diversity: 0.2
<SOS>Ur cash or a chat wid ur mobile 11mths ? Call 09061221066 from landline. Claim your mobile 10 mths ? Call 08712405020.
----- diversity: 0.6
<SOS>Ur cash or a chat just reply YES-434 or NOW! Only 10p per minutes to 80182 to contract mobile No 07786200117
----- diversity: 1.2
<SOS>Ur cash-in now cust10p/Msgr88Q!
Epoch 70/500
  2560/101492 [..............................] - ETA: 5s - loss: 0.4335 - acc: 0.8441

101492/101492 [==============================] - 6s 56us/step - loss: 0.4686 - acc: 0.8283
Epoch 71/500
101492/101492 [==============================] - 6s 56us/step - loss: 0.4672 - acc: 0.8279
Epoch 72/500
101492/101492 [==============================] - 6s 56us/step - loss: 0.4662 - acc: 0.8289
Epoch 73/500
 62976/101492 [=================>............] - ETA: 2s - loss: 0.4556 - acc: 0.8320

KeyboardInterrupt: ignored